In [ ]:
# # LangChain Looker SQL Agent - Comprehensive Tests
# 
# This notebook provides a comprehensive setup and testing environment for the custom
# LangChain Looker SQL Agent. It covers:
# 1. Environment setup (Java, Python packages).
# 2. Configuration of Looker and LLM credentials.
# 3. Initialization and manual testing of the `LookerSQLDatabase` component.
# 4. Creation of the LangChain agent using `LookerSQLToolkit`.
# 5. Basic conversational tests with the agent.
# 6. Advanced tests focusing on Looker-specific SQL syntax (measures with `AGGREGATE()`, etc.).
# 
# **File Structure Expectation:**
# ```
# project_root/
# ├── .env
# ├── langchain_looker_sql_agent.py
# ├── notebooks/
# │   └── looker_langchain_sql_agent_tests.ipynb  <-- This notebook
# └── drivers/
#     └── looker-jdbc.jar (or avatica-version-looker.jar)
# ```
# 
# **Prerequisites (within your execution environment, e.g., Vertex AI Notebook VM):**
# *   Java (JRE/JDK) version 11 or newer.
# *   The Looker Avatica JDBC driver JAR file (ensure `LOOKER_JDBC_DRIVER_PATH` in `.env` points to it).
# *   Python 3.8+.
# *   Required Python packages (see Cell 1.1).

# ## Phase 1: Environment Setup & Initial Checks

# ### Cell 1.1: Install/Upgrade Dependencies (Run Once if Needed)
# *If running for the first time or after a long break, uncomment and run this cell.*
# *You MUST restart the kernel after these installations if they made changes.*

# %%
# # Ensure Java is installed (example for Debian/Ubuntu based systems)
# !sudo apt-get update && sudo apt-get install -y openjdk-11-jdk --no-install-recommends
# !java -version # Verify Java installation

# # Install/Upgrade Python packages
# !pip install -U langchain langchain-core langchain-openai langchain-community jaydebeapi python-dotenv ipywidgets JPype1 tabulate

# print("Dependencies check/installation step complete. If Java was installed or major pip changes occurred, PLEASE RESTART THE KERNEL NOW.")

# %% [markdown]
# ### Cell 1.2: Core Imports, Logger, Path Setup, Environment Loading, and Configuration Validation
# *This single cell handles all initial Python setup.*

# %%
import os
import logging 
from dotenv import load_dotenv 
import sys 

# --- 1. Configure basic logging FIRST ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__) # Logger for this notebook
# To see more detailed logs from your custom agent module during notebook execution:
# logging.getLogger("langchain_looker_agent").setLevel(logging.DEBUG) 
# Or for this notebook's logger:
# logger.setLevel(logging.DEBUG)

# --- 2. Determine project_root and add src to sys.path ---
# This logic assumes your notebook is in a subdirectory (e.g., 'notebooks') 
# of the main project root where 'src' and '.env' reside.
project_root = "" 
current_notebook_dir = os.getcwd() 

try:
    project_root = os.path.abspath(os.path.join(current_notebook_dir, os.pardir)) 
    src_dir_path = os.path.join(project_root, "src")

    if project_root not in sys.path:
        sys.path.insert(0, project_root) # Makes .env findable if it's at root
        logger.info(f"Added project root to sys.path: {project_root}")
    
    if src_dir_path not in sys.path: # Makes src/langchain_looker_agent findable
        sys.path.insert(0, src_dir_path)
        logger.info(f"Added src directory to sys.path: {src_dir_path}")
    
    # Verify by trying to list contents of expected package directory
    # test_pkg_path = os.path.join(src_dir_path, "langchain_looker_agent")
    # if os.path.isdir(test_pkg_path):
    #     logger.info(f"Verified package directory exists: {test_pkg_path} with contents: {os.listdir(test_pkg_path)}")
    # else:
    #     logger.warning(f"Package directory {test_pkg_path} NOT found. Imports might fail if not installed via -e .")

except Exception as e:
    logger.error(f"Error determining project/src root or modifying sys.path: {e}. Using current dir as fallback project_root.")
    project_root = current_notebook_dir 
    if project_root not in sys.path: sys.path.insert(0, project_root)

# --- 3. Load environment variables from .env file (expected in project_root) ---
dotenv_filepath = os.path.join(project_root, '.env') 
if os.path.exists(dotenv_filepath): 
    load_dotenv(dotenv_path=dotenv_filepath) 
    logger.info(f".env file loaded successfully from: {dotenv_filepath}")
else: 
    logger.error(f".env file not found at {dotenv_filepath}. Ensure it's in the project root: {project_root}")
    raise FileNotFoundError(f".env file not found at {dotenv_filepath}")

# --- 4. Custom Looker agent components (now imported from the package) ---
try:
    from langchain_looker_agent import LookerSQLDatabase, LookerSQLToolkit, create_looker_sql_agent 
    logger.info("Custom Looker agent classes imported successfully from 'langchain_looker_agent' package.")
except ImportError as e:
    logger.error(f"Error importing from 'langchain_looker_agent': {e}. Ensure the package is installed (e.g. `pip install -e .` from project root) and sys.path is correct.", exc_info=True)
    raise 

# --- 5. LangChain specific imports ---
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor 
logger.info("Core LangChain components (ChatOpenAI, ConversationBufferMemory, AgentExecutor) imported.")

# --- 6. Fetch and Validate ALL Environment Variables needed (rest of your Cell 1.3 content) ---
# (The rest of your variable fetching and validation logic from your notebook's original Cell 1.3 goes here)
# Make sure LOOKER_JDBC_DRIVER_PATH is resolved correctly relative to project_root if it's a relative path in .env
LOOKER_INSTANCE_URL = os.getenv("LOOKER_INSTANCE_URL")
LOOKML_MODEL_NAME = os.getenv("LOOKML_MODEL_NAME")   
LOOKER_CLIENT_ID = os.getenv("LOOKER_CLIENT_ID")
LOOKER_CLIENT_SECRET = os.getenv("LOOKER_CLIENT_SECRET")
_LOOKER_JDBC_DRIVER_PATH_FROM_ENV = os.getenv("LOOKER_JDBC_DRIVER_PATH")

# Resolve JDBC driver path relative to project_root if not absolute
if _LOOKER_JDBC_DRIVER_PATH_FROM_ENV and not os.path.isabs(_LOOKER_JDBC_DRIVER_PATH_FROM_ENV):
    LOOKER_JDBC_DRIVER_PATH = os.path.join(project_root, _LOOKER_JDBC_DRIVER_PATH_FROM_ENV)
    logger.info(f"Resolved relative JDBC driver path from .env to: {LOOKER_JDBC_DRIVER_PATH}")
else:
    LOOKER_JDBC_DRIVER_PATH = _LOOKER_JDBC_DRIVER_PATH_FROM_ENV

LOOKER_HOST_DISPLAY = os.getenv("LOOKER_HOST", LOOKER_INSTANCE_URL) 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

if "JAVA_HOME" not in os.environ or not os.path.isdir(os.environ.get("JAVA_HOME","")) or "bin/java" in os.environ.get("JAVA_HOME",""):
    java_home_correct_path = "/usr/lib/jvm/java-11-openjdk-amd64" 
    if os.path.exists(java_home_correct_path) and os.path.isdir(java_home_correct_path):
        os.environ["JAVA_HOME"] = java_home_correct_path; logger.info(f"JAVA_HOME set to: {os.environ['JAVA_HOME']}")
    else: logger.warning(f"JAVA_HOME incorrect/not set, default path '{java_home_correct_path}' not found.")
else: logger.info(f"JAVA_HOME is correctly set to: {os.environ['JAVA_HOME']}")

missing_vars = []
if not LOOKER_INSTANCE_URL: missing_vars.append("LOOKER_INSTANCE_URL")
if not LOOKML_MODEL_NAME: missing_vars.append("LOOKML_MODEL_NAME")
# ... (all your other missing_vars checks)
if not LOOKER_JDBC_DRIVER_PATH: missing_vars.append("LOOKER_JDBC_DRIVER_PATH (after potential resolution)")
if not OPENAI_API_KEY: missing_vars.append("OPENAI_API_KEY")

if missing_vars:
    err_msg = f"Required environment variables are missing: {', '.join(missing_vars)}"
    logger.critical(err_msg); raise ValueError(err_msg)
if not LOOKER_JDBC_DRIVER_PATH or not os.path.exists(LOOKER_JDBC_DRIVER_PATH): # Check resolved path
    err_msg = f"Looker JDBC Driver not found at resolved path: {LOOKER_JDBC_DRIVER_PATH}."
    logger.critical(err_msg); raise FileNotFoundError(err_msg)
    
logger.info("All configurations loaded and validated.")
logger.info(f"  Using Looker Instance URL: {LOOKER_INSTANCE_URL}")
logger.info(f"  Using LookML Model Name: {LOOKML_MODEL_NAME}")
logger.info(f"  Actual JDBC Driver Path Used: {LOOKER_JDBC_DRIVER_PATH}")
logger.info(f"  JAVA_HOME: {os.getenv('JAVA_HOME', 'Not Set - Check Warnings!')}")

# The cell that initializes LookerSQLDatabase should also use the resolved path:
# In Phase 2 of notebook:
# db = LookerSQLDatabase(
#     ...
#     jdbc_driver_path=LOOKER_JDBC_DRIVER_PATH, # This is now the resolved full path
#     ...
# )



# ## Phase 2: Initialize `LookerSQLDatabase`

# %%
db = None 
try:
    logger.info("Attempting to initialize LookerSQLDatabase...")
    # Construct absolute path for JDBC driver if it's relative from .env
    jdbc_driver_full_path = LOOKER_JDBC_DRIVER_PATH
    if not os.path.isabs(jdbc_driver_full_path):
        jdbc_driver_full_path = os.path.join(project_root, jdbc_driver_full_path)
        logger.info(f"Resolved relative JDBC driver path to: {jdbc_driver_full_path}")
    
    if not os.path.exists(jdbc_driver_full_path):
        err_msg = f"Looker JDBC Driver still not found at resolved path: {jdbc_driver_full_path}."
        logger.critical(err_msg); raise FileNotFoundError(err_msg)

    db = LookerSQLDatabase(
        looker_instance_url=LOOKER_INSTANCE_URL, 
        lookml_model_name=LOOKML_MODEL_NAME, 
        client_id=LOOKER_CLIENT_ID,
        client_secret=LOOKER_CLIENT_SECRET,
        jdbc_driver_path=jdbc_driver_full_path, # Use resolved full path
        sample_rows_in_table_info=2, 
    )
    if db._connection: logger.info("LookerSQLDatabase initialized and connection appears established.")
    else: logger.warning("LookerSQLDatabase initialized BUT internal _connection is None (unexpected).")
except Exception as e:
    logger.error(f"CRITICAL: LookerSQLDatabase initialization failed: {e}", exc_info=True)

# ## Phase 3: Manual `LookerSQLDatabase` Tests
# *These tests directly use the `db` object to verify core database interaction capabilities.*

# %%
if db and db._connection: 
    try:
        logger.info("\n--- Manual LookerSQLDatabase Method Tests ---")
        
        # Test 1: Get usable table names (Explores)
        logger.info("Test 1: Getting usable Explores...")
        explore_names_from_db = list(db.get_usable_table_names()) # Renamed to avoid clash with notebook-level var
        if explore_names_from_db:
            logger.info(f"SUCCESS: Usable Explores found: {explore_names_from_db[:5]}{'...' if len(explore_names_from_db) > 5 else ''} (Total: {len(explore_names_from_db)})")
            
            # Use the first explore for subsequent schema and query tests
            first_explore_for_manual_test = explore_names_from_db[0]
            logger.info(f"Will use Explore '{first_explore_for_manual_test}' for some manual tests.")

            # Test 2: Get table info (schema for one Explore)
            logger.info(f"\nTest 2: Getting schema for Explore '{first_explore_for_manual_test}'...")
            schema_info_manual = db.get_table_info([first_explore_for_manual_test])
            print("\nSCHEMA INFO (First Explore):\n", schema_info_manual) 
            logger.info(f"SUCCESS: Schema retrieved for '{first_explore_for_manual_test}'.")

            # Test 3: Run a manual COUNT(*) SQL query
            # Model name should be correctly fetched from environment variables by now
            model_name_for_manual_query = LOOKML_MODEL_NAME 
            manual_count_query = f'SELECT COUNT(*) AS record_count FROM `{model_name_for_manual_query}`.`{first_explore_for_manual_test}`'
            logger.info(f"\nTest 3: Running manual COUNT(*) query: {manual_count_query}")
            count_query_result = db.run(manual_count_query) 
            logger.info(f"Manual COUNT(*) Query Result:\n{count_query_result}")
            if "Error" not in count_query_result and "java.sql.SQLException" not in count_query_result:
                logger.info("SUCCESS: Manual COUNT(*) query executed successfully.")
            else:
                logger.error(f"Manual COUNT(*) query execution FAILED. Result: {count_query_result}")

            # --- Test 4: Manual AGGREGATE() query ---
            # IMPORTANT: You MUST adapt 'explore_for_agg_test' and 'measure_for_agg_test'
            # to valid names from your Looker model and one of the explores listed above.
            # This example uses placeholders or common names; verify them against your schema output.
            
            explore_for_agg_test = "web_sessions_fact" # Example: use one of your known explores
            # Check if this explore is actually available, otherwise pick another
            if explore_for_agg_test not in explore_names_from_db and explore_names_from_db:
                logger.warning(f"Explore '{explore_for_agg_test}' not found for AGGREGATE test, using '{explore_names_from_db[0]}' instead. Update measure name accordingly.")
                explore_for_agg_test = explore_names_from_db[0]
                # YOU WOULD NEED TO KNOW A VALID MEASURE FOR THIS FALLBACK EXPLORE
                measure_for_agg_test = "`some_view.some_measure`" # Placeholder - MUST BE UPDATED if explore changes
            elif explore_for_agg_test in explore_names_from_db:
                # Define a known measure from the chosen explore_for_agg_test
                # Example for 'web_sessions_fact': `web_events_fact.total_page_views` or `web_sessions_fact.count`
                if explore_for_agg_test == "web_sessions_fact":
                    measure_for_agg_test = "`web_events_fact.total_page_views`" 
                    # Or try: measure_for_agg_test = "`web_sessions_fact.count`"
                else:
                    # If you chose a different explore, you MUST provide a valid measure for it here
                    measure_for_agg_test = "`some_view_in_that_explore.some_measure_name`" 
                    logger.warning(f"Using placeholder measure '{measure_for_agg_test}' for explore '{explore_for_agg_test}'. Please verify this is a valid measure.")

            if explore_for_agg_test and measure_for_agg_test != "`some_view_in_that_explore.some_measure_name`": # Check if it's not the placeholder
                manual_agg_query = f"SELECT AGGREGATE({measure_for_agg_test}) AS aggregated_measure FROM `{model_name_for_manual_query}`.`{explore_for_agg_test}`"
                logger.info(f"\nTest 4 (Manual): Running AGGREGATE query: {manual_agg_query}")
                try:
                    agg_result = db.run(manual_agg_query)
                    logger.info(f"Manual AGGREGATE Query Result:\n{agg_result}")
                    if "Error" not in agg_result and "java.sql.SQLException" not in agg_result:
                        logger.info("SUCCESS: Manual AGGREGATE query executed successfully.")
                    else:
                        logger.error(f"Manual AGGREGATE query FAILED. Result: {agg_result}")
                except Exception as e_agg:
                    logger.error(f"Exception during manual AGGREGATE query: {e_agg}", exc_info=True)
            else:
                logger.warning(f"Skipping manual AGGREGATE test: Either explore '{explore_for_agg_test}' is not valid or measure '{measure_for_agg_test}' is a placeholder and needs to be updated with a real measure name.")

        else:
            logger.warning("MANUAL TESTS: No Explores found. Cannot proceed with schema or query tests.")
            
    except Exception as e:
        logger.error(f"ERROR during manual database tests: {e}", exc_info=True)
else:
    logger.warning("Skipping manual database tests: `LookerSQLDatabase` (db) is not available or not connected.")

# ## Phase 4: Initialize LLM, Memory, and `LookerSQLToolkit`

# %%
llm = None
memory = None
looker_toolkit = None

if db and db._connection: 
    try:
        logger.info("Initializing LLM...")
        llm = ChatOpenAI(model="gpt-4o", temperature=0) 
        logger.info(f"LLM initialized: {llm.model_name}")

        logger.info("Initializing ConversationBufferMemory...")
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        logger.info("ConversationBufferMemory initialized.")

        logger.info("Initializing LookerSQLToolkit...")
        looker_toolkit = LookerSQLToolkit(db=db)
        logger.info("LookerSQLToolkit initialized.")
    except Exception as e:
        logger.error(f"Error initializing LLM, Memory, or Toolkit: {e}", exc_info=True)
else:
    logger.warning("Skipping LLM, Memory, and Toolkit initialization as database connection failed.")

# ## Phase 5: Create the `AgentExecutor`

# %%
agent_executor = None
if llm and looker_toolkit and memory:
    try:
        logger.info("Creating the Looker SQL Agent Executor...")
        agent_executor = create_looker_sql_agent(
            llm=llm,
            toolkit=looker_toolkit,
            verbose=True, 
            top_k=10, 
            agent_executor_kwargs={
                "memory": memory,
                "handle_parsing_errors": True, 
                "max_iterations": 10 # Increased for potentially complex reasoning
            }
        )
        logger.info("Looker SQL Agent Executor created successfully.")
    except Exception as e:
        logger.error(f"Failed to create Looker SQL Agent Executor: {e}", exc_info=True)
else:
    logger.warning("Skipping Agent Executor creation due to missing LLM, Toolkit, or Memory.")

# ## Phase 6: Pre-determined Agent Tests (Basic and Advanced)
# *This section runs a series of pre-defined questions to test core agent functionality,
# including listing explores, getting schema, basic queries, and handling of Looker measures.*

# %%
def run_agent_test_case(question: str, agent_exec: AgentExecutor, test_name: str):
    """Helper function to run a single test case for the agent."""
    if not agent_exec:
        logger.warning(f"AGENT TEST SKIPPED ({test_name}): Agent Executor not initialized.")
        print(f"\n--- SKIPPING TEST: {test_name} (Agent not initialized) ---")
        print(f"QUESTION: {question}")
        return "Agent not initialized."
        
    logger.info(f"\n--- Running Agent Test: {test_name} ---")
    print(f"\n--- Running Agent Test: {test_name} ---")
    print(f"QUESTION: {question}")
    
    # Persona/task instruction for the agent for this specific query
    # Note: The main SQL syntax rules are already in the agent's system prompt.
    instruction_prefix = f"""You are a helpful data analyst for Rittman Analytics, querying data through Looker.
The user is asking about data available in our Looker instance via the model '{LOOKML_MODEL_NAME}'.
If the question involves a calculation on a LookML measure (often indicated by `MEASURE<TYPE>` in the schema), 
ensure you wrap that measure with `AGGREGATE()`. For example, `SELECT AGGREGATE(\`view.measure_field\`) ...`.
Question: """
    
    full_question_for_agent = instruction_prefix + question

    try:
        # AgentExecutor with memory handles chat_history implicitly if memory was passed during its creation.
        # For isolated tests, we still pass an empty chat_history to ensure the prompt is satisfied.
        response = agent_exec.invoke({
            "input": full_question_for_agent,
            "chat_history": [] # Provide for stateless test invocation
        }) 
        output = response.get("output", "No 'output' field in agent response.")
        print(f"\nAGENT RESPONSE for '{test_name}':\n{output}")
        logger.info(f"Agent response for '{test_name}': {output[:500]}{'...' if len(output) > 500 else ''}")
        return output
    except Exception as e:
        logger.error(f"ERROR invoking agent for test '{test_name}' (Question: '{question}'): {e}", exc_info=True)
        print(f"ERROR during agent invocation for '{test_name}': {e}")
        return f"Error: {e}"
    finally:
        print("-" * 70)

# %%
if agent_executor:
    logger.info("\n--- Executing Pre-determined Agent Tests ---")

    # --- Test Case 1: List available Explores ---
    run_agent_test_case(
        question="List all available tables I can query.",
        agent_exec=agent_executor,
        test_name="List Explores"
    )

    # --- Determine an Explore to use for subsequent tests ---
    explore_for_subsequent_tests = None
    if db and db._connection:
        try:
            available_explores = list(db.get_usable_table_names())
            if available_explores:
                # Prefer 'web_sessions_fact' or 'chart_of_accounts_dim' if available, else first one
                if "web_sessions_fact" in available_explores:
                    explore_for_subsequent_tests = "web_sessions_fact"
                elif "chart_of_accounts_dim" in available_explores:
                    explore_for_subsequent_tests = "chart_of_accounts_dim"
                else:
                    explore_for_subsequent_tests = available_explores[0]
                logger.info(f"Will use Explore '{explore_for_subsequent_tests}' for subsequent targeted tests.")
            else:
                logger.warning("No usable Explores found by db object; some tests might be skipped or use placeholders.")
        except Exception as e:
            logger.error(f"Could not dynamically get Explore names for tests: {e}")
    
    if not explore_for_subsequent_tests:
        logger.warning("explore_for_subsequent_tests is not set. Subsequent tests might fail if they rely on it.")
        explore_for_subsequent_tests = "your_placeholder_explore" # Fallback to a placeholder

    # --- Test Case 2: Describe a specific Explore ---
    run_agent_test_case(
        question=f"Describe the table named '{explore_for_subsequent_tests}'. What are its columns and provide a few sample rows if possible?",
        agent_exec=agent_executor,
        test_name=f"Describe Explore '{explore_for_subsequent_tests}'"
    )

    # --- Test Case 3: Simple COUNT query ---
    run_agent_test_case(
        question=f"How many records are in the '{explore_for_subsequent_tests}' table?",
        agent_exec=agent_executor,
        test_name=f"Count Records in '{explore_for_subsequent_tests}'"
    )

    # --- Advanced Tests: Adapt these field names to YOUR Looker model ---
    # Find these in the schema output from Test Case 2 or Looker UI.
    # These are examples based on your previous logs for 'web_sessions_fact'
    # If explore_for_subsequent_tests is different, these MUST be updated.
    
    measure_field_for_test = "`web_events_fact.total_page_views`" # Example, ensure this is a valid measure in explore_for_subsequent_tests
    dimension_field_for_test = "`web_sessions_fact.device_category`" # Example dimension
    dimension_value_for_test = "desktop" # Example value for the dimension

    if explore_for_subsequent_tests == "chart_of_accounts_dim": # Example of adapting if explore changes
        measure_field_for_test = "`general_ledger_fact.net_amount`" # Assuming this is a measure in chart_of_accounts_dim
        dimension_field_for_test = "`chart_of_accounts_dim.account_class`"
        dimension_value_for_test = "ASSET" # A plausible value
        logger.info(f"Adapting advanced test fields for explore: {explore_for_subsequent_tests}")


    # --- Test Case 4: Query a LookML Measure using AGGREGATE() ---
    run_agent_test_case(
        question=f"What is the total of measure {measure_field_for_test} from the {explore_for_subsequent_tests} Explore?",
        agent_exec=agent_executor,
        test_name=f"Aggregate Measure {measure_field_for_test}"
    )

    # --- Test Case 5: Measure aggregated by a dimension ---
    run_agent_test_case(
        question=f"Show me the total {measure_field_for_test} grouped by {dimension_field_for_test} from the {explore_for_subsequent_tests} Explore.",
        agent_exec=agent_executor,
        test_name=f"Aggregate Measure by Dimension"
    )
    
    # --- Test Case 6: Measure with a WHERE clause on a dimension ---
    run_agent_test_case(
        question=f"What is the total {measure_field_for_test} for {dimension_field_for_test} = '{dimension_value_for_test}' from the {explore_for_subsequent_tests} Explore?",
        agent_exec=agent_executor,
        test_name=f"Aggregate Measure with Filter"
    )

    # --- Test Case 7: Query that should NOT use JOIN, subquery, or window function ---
    run_agent_test_case(
        question=f"List the top 3 {dimension_field_for_test} by total {measure_field_for_test} from {explore_for_subsequent_tests}, ordered descending by that total.",
        agent_exec=agent_executor,
        test_name="Top N without Window Functions"
    )

else:
    logger.error("Agent Executor not initialized. Cannot run pre-determined agent tests.")
    print("ERROR: Agent Executor could not be initialized. Please review setup cells for errors.")


# ## Phase 7: Cleanup (Optional) 
# *(Renumbered from Phase 9 as the interactive loop was removed from this notebook)*

# %%
if db:
    try:
        logger.info("Attempting to close Looker database connection...")
        db.close()
    except Exception as e:
        logger.error(f"Error closing Looker connection: {e}", exc_info=True)
else:
    logger.info("No database connection (db object) to close or it was not initialized.")

logger.info("--- End of Looker SQL Agent Comprehensive Tests Notebook ---")